In [1]:
import os
import json
import swifter
import numpy as np

from pandas.io.json import json_normalize
from ast import literal_eval
import time
import datetime

from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

In [153]:
a = time.time()
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
COLUMNS = ['fullVisitorId', 'date', 'totals']
data = pd.read_csv(r'raw_data/train_v2.csv', converters={column: json.loads for column in JSON_COLUMNS},dtype={'fullVisitorId': 'str'}, usecols = COLUMNS, parse_dates = ['date'])
data.head()
print(time.time() - a)

110.80548810958862


In [154]:
data.dtypes

date             datetime64[ns]
fullVisitorId            object
totals                   object
dtype: object

In [18]:
date2 = '20170101'
to_date = datetime.datetime.strptime(date2, "%Y%m%d")

future1 = to_date + datetime.timedelta(days=46)
future2 = to_date + datetime.timedelta(days=108)
print(future1)

2017-02-16 00:00:00


In [19]:
data2 = data[(data['date']>=future1) & (data['date']<=future2)]

In [9]:
df0 = pd.read_csv(r'predictive_alok/train_data_class/train0.csv')
df0.drop(['LogRevenue', 'Purchase'], axis = 1)

,fullVisitorId,visits,(Other),Affiliates,Direct,Display,Organic Search,Paid Search,Referral,Social,...,trafSrc_isTrueDir_isNULL,trafSrc_isTrueDir_True,trafSrc_kw_others,trafSrc_kw_isNULL,trafSrc_medium_org,trafSrc_medium_ref,trafSrc_medium_isNULL,trafSrc_medium_cpc,trafSrc_medium_aff,trafSrc_medium_cpm
0,10278554503158,1,0,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,1,0,0
1,20424342248747,1,0,0,0,0,1,0,0,0,...,1,0,0,1,0,1,0,0,0,0
2,5103959234087,1,0,0,0,0,1,0,0,0,...,1,0,0,1,0,1,0,0,0,0
3,93957001069502,1,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,114156543135683,1,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377181,9999761280717362235,2,0,0,0,0,0,2,0,0,...,2,0,0,2,0,0,0,0,0,0
377182,9999774833256469931,1,0,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
377183,9999801229043043045,1,0,0,1,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,0
377184,9999887420016307570,1,0,0,0,0,1,0,0,0,...,1,0,1,0,1,0,0,0,0,0


In [29]:
df0.head()

,fullVisitorId,LogRevenue,Purchase,visits,(Other),Affiliates,Direct,Display,Organic Search,Paid Search,...,trafSrc_isTrueDir_isNULL,trafSrc_isTrueDir_True,trafSrc_kw_others,trafSrc_kw_isNULL,trafSrc_medium_org,trafSrc_medium_ref,trafSrc_medium_isNULL,trafSrc_medium_cpc,trafSrc_medium_aff,trafSrc_medium_cpm
0,10278554503158,0.0,0,1,0,0,0,0,1,0,...,1,0,1,0,0,0,0,1,0,0
1,20424342248747,0.0,0,1,0,0,0,0,1,0,...,1,0,0,1,0,1,0,0,0,0
2,5103959234087,0.0,0,1,0,0,0,0,1,0,...,1,0,0,1,0,1,0,0,0,0
3,93957001069502,0.0,0,1,0,0,0,0,1,0,...,1,0,0,1,0,0,0,0,0,0
4,114156543135683,0.0,0,1,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0


In [189]:
def target_variable(data, date1, date2, file):
#     print(date1, date2)
    from_date = datetime.datetime.strptime(date1, "%Y%m%d")
    to_date = datetime.datetime.strptime(date2, "%Y%m%d")
    future1 = to_date + datetime.timedelta(days=46)
    future2 = to_date + datetime.timedelta(days=108)    
#     print('\n')
#     print(from_date, to_date, future1, future2)
    
    data1 = data[(data['date']>=from_date) & (data['date']<=to_date)]
    data1 = data1[['fullVisitorId']].drop_duplicates()
    future = data[(data['date']>=future1) & (data['date']<=future2)].reset_index()
    visitorID = future.fullVisitorId.unique()
    
    JSON_COLUMNS = ['totals']
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(future[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        future = future.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True, how = 'left')
    
    returned_customers = future[future['fullVisitorId'].isin(visitorID)]
    df_target = returned_customers[['fullVisitorId', 'totals.transactionRevenue']]
    df_target['totals.transactionRevenue'] = df_target['totals.transactionRevenue'].astype(float)
    df_target = df_target.fillna(0)
    df_target = df_target.groupby('fullVisitorId').agg({'totals.transactionRevenue':'sum'})
    df_target['LogRevenue_y'] = np.log(df_target['totals.transactionRevenue'].astype(float) + 1)
    df_target['Purchase_y'] = df_target['LogRevenue_y'].swifter.apply(flag)
    df_target = df_target[['LogRevenue_y', 'Purchase_y']].reset_index()
    
    final  = pd.merge(data1, df_target, on = 'fullVisitorId', how = 'left')
    final = final.fillna(0)
    
    path = 'predictive_alok/train_data_class/train' + file + '.csv'
    path2 = 'predictive_alok/train_data_class/final_final/train' + file + '.csv'
    df0 = pd.read_csv(path,dtype={'fullVisitorId': 'str'})
    final = pd.merge(final, df0, on = 'fullVisitorId')
    print(final.groupby('Purchase').size())
    final.to_csv(path2, index = False)
    print('done!')
    return 1 

In [190]:
training_from = [20160801,20160901, 20161001, 20161101, 20161201, 20170101, 20170201, 20170301, 20170401, 20170501, 20170601, 20170701, 20170801] 
training_to = [20170115,20170215,  20170315,  20170415,  20170515,  20170615,  20170715,  20170815,  20170915,  20171015,  20171115,  20171215,  20180115]  

for i in range(13):
    target_variable(data, str(training_from[i]), str(training_to[i]), str(i))


Purchase
0    375492
1      1694
dtype: int64
done!



Purchase
0    370255
1      1711
dtype: int64
done!



Purchase
0    363613
1      1749
dtype: int64
done!



Purchase
0    338446
1      1532
dtype: int64
done!



Purchase
0    292514
1      1020
dtype: int64
done!


...


Purchase
0    304415
1      1345
dtype: int64
done!



Purchase
0    328467
1      1169
dtype: int64
done!



Purchase
0    353409
1      1066
dtype: int64
done!



Purchase
0    383888
1      1058
dtype: int64
done!



Purchase
0    388599
1      1029
dtype: int64
done!


In [80]:
#date1 = '20160801'
#from_date = datetime.datetime.strptime(date1, "%Y%m%d") #min(data$date)+168*(k-1)
#to_date = from_date + datetime.timedelta(days=168)    #min(data$date) + 168*k
#future1 = to_date + datetime.timedelta(days=46)  #min(data$date) + 168*k + 46
#future2 = to_date + datetime.timedelta(days=108) #min(data$date) + 168*k + 46 +62
#

#tf = data[(data['date']>=from_date) & (data['date'] < to_date)]

#tf_fvid_data = data[(data['date']>=future1) & (data['date'] < future2)]
#tf_fvid = tf_fvid_data[['fullVisitorId']].drop_duplicates()


#tf_returned = tf[tf['fullVisitorId'].isin(tf_fvid['fullVisitorId'])]
#tf_returned_visitorID = tf_returned.fullVisitorId.unique()


tf_tst = data[(data['fullVisitorId'].isin(tf_returned_visitorID)) & (data['date']>=future1) & (data['date'] < future2)]
########################################################################################--- okay


# data1 = data1[['fullVisitorId']].drop_duplicates()
# future = data[(data['date']>=future1) & (data['date']<=future2)]
# visitorID = future.fullVisitorId.unique()

In [6]:
path = 'predictive_alok/train_data_class/final_final/train0.csv'
# path2 = 'predictive_alok/train_data_class/final_final/train' + file + '.csv'
df0 = pd.read_csv(path,dtype={'fullVisitorId': 'str'})
df0.groupby('Purchase_y').size()
# df0.columns

Purchase_y
0.0    377036
1.0       150
dtype: int64

In [119]:
def add_brackets(x):
    return list(x)

tf_tst['totals_1'] = tf_tst['totals'].apply(add_brackets)
tf_tst.head()

,date,fullVisitorId,totals,totals_1
13084,2017-03-12,6018775317735347795,"{'visits': '1', 'hits': '1', 'pageviews': '1',...","[visits, hits, pageviews, bounces]"
13090,2017-03-12,4912143557358780799,"{'visits': '1', 'hits': '1', 'pageviews': '1',...","[visits, hits, pageviews, bounces]"
13099,2017-03-12,850297961947879420,"{'visits': '1', 'hits': '1', 'pageviews': '1',...","[visits, hits, pageviews, bounces]"
13106,2017-03-12,4950398280284706032,"{'visits': '1', 'hits': '1', 'pageviews': '1',...","[visits, hits, pageviews, bounces]"
13131,2017-03-12,7648310613375845839,"{'visits': '1', 'hits': '1', 'pageviews': '1',...","[visits, hits, pageviews, bounces]"


In [150]:
future = tf_tst.reset_index()
JSON_COLUMNS = ['totals']
for column in JSON_COLUMNS:
    column_as_df = json_normalize(future[column])
    column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
    future = future.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True, how = 'left')

In [160]:
df_target = future
df_target['totals.transactionRevenue'] = df_target['totals.transactionRevenue'].astype(float)
df_target = df_target.fillna(0)
df_target = df_target.groupby('fullVisitorId').agg({'totals.transactionRevenue':'sum'})
df_target['LogRevenue'] = np.log(df_target['totals.transactionRevenue'].astype(float) + 1)
df_target['Purchase'] = df_target['LogRevenue'].swifter.apply(flag)
df_target = df_target[['LogRevenue', 'Purchase']].reset_index()

In [164]:
df_target[df_target['LogRevenue'] != 0]

,fullVisitorId,LogRevenue,Purchase
16,0128830210468179660,16.810743,1
20,0164339786303841762,18.197412,1
24,0218721670797647024,18.979325,1
34,0290663072707523580,19.228718,1
38,0328080880890336475,17.936010,1
...,...,...,...
1755,9467619663210756890,19.517959,1
1802,9697396763466620407,18.797884,1
1829,9826816980924031810,17.375988,1
1842,9909403981445501787,18.055397,1


In [161]:
future[future['fullVisitorId'] == '0128830210468179660']

,index,date,fullVisitorId,totals_1,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.timeOnSite,totals.newVisits,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue
1948,671720,2017-04-03,0128830210468179660,"[visits, hits, pageviews, timeOnSite, transact...",1,21,18,NaN,1673,NaN,1,19990000.0,28990000


In [135]:
#df_target[df_target['LogRevenue'] != 0.0]

# json_normalize(tf_tst[tf_tst['fullVisitorId'] == '0128830210468179660'].totals)

future = tf_tst[tf_tst['fullVisitorId'] == '0128830210468179660']
JSON_COLUMNS = ['totals']
for column in JSON_COLUMNS:
#     column_as_df = json_normalize(data[column])
#     column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
#     data = data.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True, how = 'left')
    
    column_as_df = json_normalize(future[column])
#     column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
    future = future.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True, how = 'left')

In [137]:
column_as_df

,visits,hits,pageviews,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue
0,1,21,18,1673,1,19990000,28990000


In [136]:
future

,date,fullVisitorId,totals_1,visits,hits,pageviews,timeOnSite,transactions,transactionRevenue,totalTransactionRevenue
671720,2017-04-03,0128830210468179660,"[visits, hits, pageviews, timeOnSite, transact...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
target = target_variable(data, '20160801' , '20170315', 'predictive_alok/train_data_class/train0.csv')

In [172]:
target.groupby('Purchase').size()

Purchase
0    375492
1      1694
dtype: int64

In [159]:
target[target['fullVisitorId'] == '0128830210468179660']

,fullVisitorId,LogRevenue,Purchase,visits,(Other),Affiliates,Direct,Display,Organic Search,Paid Search,...,trafSrc_isTrueDir_isNULL,trafSrc_isTrueDir_True,trafSrc_kw_others,trafSrc_kw_isNULL,trafSrc_medium_org,trafSrc_medium_ref,trafSrc_medium_isNULL,trafSrc_medium_cpc,trafSrc_medium_aff,trafSrc_medium_cpm
270447,0128830210468179660,16.810743,1.0,1,0,0,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0


In [224]:
training_from = [20160801,20160901, 20161001, 20161101, 20161201, 20170101, 20170201, 20170301, 20170401, 20170501, 20170601, 20170701, 20170801] 
training_to = [20170115,20170215,  20170315,  20170415,  20170515,  20170615,  20170715,  20170815,  20170915,  20171015,  20171115,  20171215,  20180115]  

target_from = [20170301,20170401,20170501,20170601,20170701,20170801,20170901,20171001,20171101,20171201,20180101,20180201,20180301]
target_to = [20170501,20170601,20170701,20170801,20170901,20171001,20171101,20171201,20180101,20180201,20180301,20180401,20180501]

In [28]:
print("min date",min(data['date']))
print("max date",max(data['date']))

min date 20180501
max date 20181015


In [5]:
print("min date",min(test['date']))
print("max date",max(test['date']))

NameError: name 'test' is not defined

In [17]:
data.dtypes

channelGrouping     object
customDimensions    object
date                 int64
device              object
fullVisitorId       object
geoNetwork          object
totals              object
trafficSource       object
visitId              int64
visitNumber          int64
dtype: object

In [9]:
data['fullVisitorID'] = data['fullVisitorId'].astype(str)

In [18]:
data.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,totals,trafficSource,visitId,visitNumber
0,Organic Search,"[{'index': '4', 'value': 'APAC'}]",20180511,"{'browser': 'Chrome', 'browserVersion': 'not a...",7460955084541987166,"{'continent': 'Asia', 'subContinent': 'Souther...","{'visits': '1', 'hits': '4', 'pageviews': '3',...","{'referralPath': '(not set)', 'campaign': '(no...",1526099341,2
1,Direct,"[{'index': '4', 'value': 'North America'}]",20180511,"{'browser': 'Chrome', 'browserVersion': 'not a...",460252456180441002,"{'continent': 'Americas', 'subContinent': 'Nor...","{'visits': '1', 'hits': '4', 'pageviews': '3',...","{'referralPath': '(not set)', 'campaign': '(no...",1526064483,166
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20180511,"{'browser': 'Chrome', 'browserVersion': 'not a...",3461808543879602873,"{'continent': 'Americas', 'subContinent': 'Nor...","{'visits': '1', 'hits': '4', 'pageviews': '3',...","{'referralPath': '(not set)', 'campaign': '(no...",1526067157,2
3,Direct,"[{'index': '4', 'value': 'North America'}]",20180511,"{'browser': 'Chrome', 'browserVersion': 'not a...",975129477712150630,"{'continent': 'Americas', 'subContinent': 'Nor...","{'visits': '1', 'hits': '5', 'pageviews': '4',...","{'referralPath': '(not set)', 'campaign': '(no...",1526107551,4
4,Organic Search,"[{'index': '4', 'value': 'North America'}]",20180511,"{'browser': 'Internet Explorer', 'browserVersi...",8381672768065729990,"{'continent': 'Americas', 'subContinent': 'Nor...","{'visits': '1', 'hits': '5', 'pageviews': '4',...","{'referralPath': '(not set)', 'campaign': '(no...",1526060254,1


In [23]:
# Splitting The Dataaframes
def mungemunge(from_date, to_date):
    train = data[(data['date']>=from_date) & (data['date']<=to_date)]
    print("Date Range",min(train['date']), max(train['date']))
    print("# of Visitors",len(train.fullVisitorId.unique()))
    print("shape of training",train.shape)

#     print("Load Dataframe to expand JSON")
    training = load_df(train)
    print("Final Shape",training.shape)
    print("# of Visitors",len(training.fullVisitorId.unique()))
#     print('done!!!')
    return training

In [24]:
# Getting Jsons
def load_df(df1):
#     a = time.time()
    print('input shape:', df1.shape)
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    for column in JSON_COLUMNS:
#         print(column)
        column_as_df = json_normalize(df1[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df1 = df1.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True, how = 'left')
#     print(time.time() -a)
    print('Splitting Json Done!! final shape:', df1.shape)
    return df1

In [21]:
# Changing Datatypes
def convert_float(x):
    try:
        return float(x)
    except ValueError:
        return np.nan

def flag(x):
    if float(x) == 0:
        return 0
    else: 
        return 1

In [26]:
# Creating dataframes

def data_munge(df):
#     a = time.time()
    
    df_target = df[['fullVisitorId', 'totals.transactionRevenue']]
    df_target['totals.transactionRevenue'] = df_target['totals.transactionRevenue'].astype(float)
    df_target = df_target.fillna(0)
    df_target = df_target.groupby('fullVisitorId').agg({'totals.transactionRevenue':'sum'})
    df_target['LogRevenue'] = np.log(df_target['totals.transactionRevenue'].astype(float) + 1)
    df_target['Purchase'] = df_target['LogRevenue'].swifter.apply(flag)
    df_target = df_target[['LogRevenue', 'Purchase']].reset_index()
    
    df_channel = training[['fullVisitorId', 'channelGrouping']]
    df_channel = pd.concat((df_channel,pd.get_dummies(df_channel.channelGrouping)),1)
    df_channel = df_channel.drop('channelGrouping', axis = 1)
    df_channel = df_channel.groupby('fullVisitorId').sum().reset_index()
    
    df_visits = df[['fullVisitorId','visitNumber']]
    df_visits = df_visits.groupby('fullVisitorId').size().reset_index()
    df_visits.columns = ['fullVisitorId', 'visits']
    
    df['totals.bounces'] = df['totals.bounces'].swifter.apply(convert_float)
    df['totals.hits'] = df['totals.hits'].swifter.apply(convert_float)
    df['totals.pageviews'] = df['totals.pageviews'].swifter.apply(convert_float)
#     df['totals.sessionQualityDim'] = df['totals.sessionQualityDim'].swifter.apply(convert_float)
    df['totals.newVisits'] = df['totals.newVisits'].swifter.apply(convert_float)
    df['totals.timeOnSite'] = df['totals.timeOnSite'].swifter.apply(convert_float)
    df['totals.transactions'] = df['totals.transactions'].swifter.apply(convert_float)
    
    df_boyang = df.fillna(-1).groupby("fullVisitorId").agg({"totals.bounces":"sum","totals.hits":"sum","totals.newVisits":"sum",\
                                                            "totals.pageviews":"sum", \
#                                                             "totals.sessionQualityDim":"sum",\
                                                            "totals.timeOnSite":"sum","totals.transactions":"sum"})  
    df_boyang.reset_index(level=0, inplace=True)

    
    
    df['trafSrc_adCont_isNULL'] = np.where(df['trafficSource.adContent'].isnull(), 1, 0)
    df['trafSrc_adCont_Others'] = np.where(df['trafficSource.adContent'].isin(["First Full Auto Template Test Ad","Ad from 11/7/16",\
                                                                               "{KeyWord:Google Brand Items}","Full auto ad TEXT ONLY",\
                                                                               "LeEco_1a","JD_5a_v1","{KeyWord:Google Men's T-Shirts}",\
                                                                               "Full auto ad TEXT/NATIVE","Official Google Merchandise - Fast Shipping",\
                                                                               "free shipping","Full auto ad with Primary Color","Free Shipping!",\
                                                                               "GA Help Center","{KeyWord:Google Branded Outerwear}","Full auto ad NATIVE ONLY",\
                                                                               "Swag w/ Google Logos"]), 1, 0)
    df['trafSrc_adCont_20disc'] = np.where(df['trafficSource.adContent'] == '20% discount', 1, 0)
    df['trafSrc_adCont_GoogleMerc'] = np.where(df['trafficSource.adContent'] == 'Google Merchandise', 1, 0)
    df['trafSrc_adCont_DispAd15'] = np.where(df['trafficSource.adContent'] == 'Display Ad created 3/11/15', 1, 0)
    df['trafSrc_adCont_Ad121316'] = np.where(df['trafficSource.adContent'] == 'Ad from 12/13/16', 1, 0)
    df['trafSrc_adCont_DispAd14'] = np.where(df['trafficSource.adContent'] == 'Display Ad created 3/11/14', 1, 0)
    df['trafSrc_adCont_FullAutoAd'] = np.where(df['trafficSource.adContent'] == 'Full auto ad IMAGE ONLY', 1, 0)    
    df['trafSrc_NetType_GSearch'] = np.where(df['trafficSource.adwordsClickInfo.adNetworkType'] == 'Google Search', 1, 0)
    df['trafSrc_NetType_SrchPartners'] = np.where(df['trafficSource.adwordsClickInfo.adNetworkType'] == 'Search Partners', 1, 0)
    df['trafSrc_gcl_isNULL'] = np.where(df['trafficSource.adwordsClickInfo.gclId'].isnull(), 1, 0)    
    df['trafSrc_isVidAd'] = np.where(df['trafficSource.adwordsClickInfo.isVideoAd'].isnull(), 1, 0)    
    df['trafSrc_page_isNULL'] = np.where(df['trafficSource.adwordsClickInfo.page'].isnull(), 1, 0)
    df['trafSrc_page_1'] = np.where(df['trafficSource.adwordsClickInfo.page'] == '1', 1, 0)
    df['trafSrc_page_others'] = np.where(df['trafficSource.adwordsClickInfo.page'].isin(['2','3','4','5','7','9']), 1, 0)
    
    df_samira = df.groupby('fullVisitorId').agg({'trafSrc_adCont_isNULL': 'sum', 'trafSrc_adCont_Others': 'sum', \
                                                 'trafSrc_adCont_20disc': 'sum', 'trafSrc_adCont_GoogleMerc': 'sum', \
                                                 'trafSrc_adCont_DispAd15': 'sum', 'trafSrc_adCont_Ad121316': 'sum', \
                                                 'trafSrc_adCont_DispAd14': 'sum', 'trafSrc_adCont_FullAutoAd': 'sum', \
                                                 'trafSrc_NetType_GSearch': 'sum', 'trafSrc_NetType_SrchPartners': 'sum', \
                                                 'trafSrc_gcl_isNULL': 'sum', 'trafSrc_isVidAd': 'sum', 'trafSrc_page_isNULL': 'sum', \
                                                 'trafSrc_page_1': 'sum', 'trafSrc_page_others': 'sum'}).reset_index()
    
    df_samira.reset_index(level=0, inplace=True)
    
    
    df['trafSrc_page2_isNULL'] = np.where(df['trafficSource.adwordsClickInfo.slot'].isnull(), 1, 0)
    df['trafSrc_page2'] = np.where(df['trafficSource.adwordsClickInfo.slot'] == 'Top', 1, 0)
    df['trafSrc_page2_others'] = np.where(df['trafficSource.adwordsClickInfo.slot'] == 'RHS', 1, 0)    
    df['trafSrc_camp'] = np.where(df['trafficSource.campaign'].isin(['Data Share Promo','AW - Dynamic Search Ads Whole Site',\
                                                                     'AW - Electronics','AW - Accessories','All Products']),1,0)
    df['trafSrc_isTrueDir_isNULL'] = np.where(df['trafficSource.isTrueDirect'].isnull(), 1, 0)
    df['trafSrc_isTrueDir_True'] = np.where(df['trafficSource.isTrueDirect'] == 'True', 1, 0)
    
    df['trafSrc_kw_others'] = np.where(df['trafficSource.keyword'].isin(['water bottle','(not provided)',
           '(Remarketing/Content targeting)', '6qEhsCssdK0z36ri',
           '(automatic matching)', 'Google men', '1hZbAqLCbjwfgOH7',
           '1X4Me6ZKNV0zg-jV', 'google online merchandise',
           'google water bottle', '(User vertical targeting)',
           'google company store', 'https://www.googlemerchandisestore.com/',
           'letterman jacket ebay', 'google canada',
           'Google where is gabika clothing', 'google online shops',
           'google Merchandising kosten', 'www.google.com bag',
           'youtube youtube', 'youtube www youtube com', 'merchdise',
           'yputube', 'youtube', 'www you tope', 'youtubeshop',
           'googel store', 'google shirts buy', 'Google mens']), 1, 0)
    
    df['trafSrc_kw_isNULL'] = np.where(df['trafficSource.keyword'].isnull(),1,0)
    
    df['trafSrc_medium_org'] = np.where(df['trafficSource.medium'] == 'organic', 1, 0)
    df['trafSrc_medium_ref'] = np.where(df['trafficSource.medium'] == 'referral', 1, 0)
    df['trafSrc_medium_isNULL'] = np.where(df['trafficSource.medium'] == '(none)', 1, 0)
    df['trafSrc_medium_cpc'] = np.where(df['trafficSource.medium'] == 'cpc', 1, 0)
    df['trafSrc_medium_aff'] = np.where(df['trafficSource.medium'] == 'affiliate', 1, 0)
    df['trafSrc_medium_cpm'] = np.where(df['trafficSource.medium'] == 'cpm', 1, 0)
    
    
    df_shobhit = df.groupby('fullVisitorId').agg({'trafSrc_page2_isNULL': 'sum', 'trafSrc_page2': 'sum', 'trafSrc_page2_others': 'sum', \
                                                  'trafSrc_camp': 'sum', 'trafSrc_isTrueDir_isNULL': 'sum', 'trafSrc_isTrueDir_True': 'sum', \
                                                  'trafSrc_kw_others': 'sum', 'trafSrc_kw_isNULL': 'sum', 'trafSrc_medium_org': 'sum', \
                                                  'trafSrc_medium_ref': 'sum', 'trafSrc_medium_isNULL': 'sum', 'trafSrc_medium_cpc': 'sum', \
                                                  'trafSrc_medium_aff': 'sum','trafSrc_medium_cpm': 'sum'})
    

    df_shobhit.reset_index(level=0, inplace=True)
#     print(df_target.shape)
    df_final = pd.merge(df_target, df_visits, how = 'left', on = 'fullVisitorId')
#     print(df_final.shape)
    df_final = pd.merge(df_final, df_channel, how = 'left', on = 'fullVisitorId')
#     print(df_final.shape)
    df_final = pd.merge(df_final, df_boyang, how = 'left', on = 'fullVisitorId')
#     print(df_final.shape)
    df_final = pd.merge(df_final, df_samira, how = 'left', on = 'fullVisitorId')
#     print(df_final.shape)
    df_final = pd.merge(df_final, df_shobhit, how = 'left', on = 'fullVisitorId')
#     print(df_final.shape)
    
    print('data munge done!!')
    
    return df_final

In [27]:
def target_variable(filename, from_date, to_date):    
    df = data[(data['date']>=from_date) & (data['date']<to_date)]
    print("date range",min(df['date']), max(df['date']))
    
    df = load_df(df)
    df_target = df[['fullVisitorId', 'totals.transactionRevenue']]
    df_target['totals.transactionRevenue'] = df_target['totals.transactionRevenue'].astype(float)
    df_target = df_target.fillna(0)
    df_target = df_target.groupby('fullVisitorId').agg({'totals.transactionRevenue':'sum'})
    df_target['LogRevenue'] = np.log(df_target['totals.transactionRevenue'].astype(float) + 1)
    df_target['Purchase'] = df_target['LogRevenue'].swifter.apply(flag)
    df_target = df_target[['LogRevenue', 'Purchase']].reset_index()
    df_target.columns = ['fullVisitorId', 'LogRevenue_y', 'Purchase_y']
    
    current_df = pd.read_csv('final_data/'+filename, sep = ',')

    final = pd.merge(current_df, df_target, on = 'fullVisitorId', how = 'left')
#     final['LogRevenue_y'] = np.where(final['LogRevenue_y'].isnull(), 0, final['LogRevenue_y'])
#     final['Purchase_y'] = np.where(final['Purchase_y'] == 1, 1, 0)
    final= final.fillna(0)
#     final['LogRevenue_y'] = final['LogRevenue_y'].fillna(0)
    final.to_csv('final/'+filename, sep = ',', index = False , float_format='%f')
    print('final file characteristics')
    print(final.shape)
    print(final[['Purchase_y', 'LogRevenue_y']].sum())
    
    return final

In [224]:
training_from = [20160801,20160901, 20161001, 20161101, 20161201, 20170101, 20170201, 20170301, 20170401, 20170501, 20170601, 20170701, 20170801] 
training_to = [20170115,20170215,  20170315,  20170415,  20170515,  20170615,  20170715,  20170815,  20170915,  20171015,  20171115,  20171215,  20180115]  

target_from = [20170301,20170401,20170501,20170601,20170701,20170801,20170901,20171001,20171101,20171201,20180101,20180201,20180301]
target_to = [20170501,20170601,20170701,20170801,20170901,20171001,20171101,20171201,20180101,20180201,20180301,20180401,20180501]

In [158]:
train1[['fullVisitorId', 'LogRevenue']].to_csv('final_data/test.csv', index = False, float_format='%f')

In [229]:
for i in range(13):
    x = time.time()
    print('Iteration:',i)
    training = mungemunge(training_from[i], training_to[i])
    train1 = data_munge(training)
    print(train1.groupby('Purchase').size())
    train1.to_csv('final_data/train' + str(i) + '.csv', index = False, float_format='%f')
    print('\n')
    final1 = target_variable('train' + str(i) + '.csv', target_from[i], target_to[i])
    
    print(time.time() - a)
    print('Iteration Done')

Iteration: 0
Date Range 20160801 20170115
# of Visitors 377186
shape of training (465858, 10)
input shape: (465858, 10)
Splitting Json Done!! final shape: (465858, 56)
Final Shape (465858, 56)
# of Visitors 377186



data munge done!!
Purchase
0    375492
1      1694
dtype: int64


date range 20170301 20170430
input shape: (137057, 10)
Splitting Json Done!! final shape: (137057, 55)



final file characteristics
(377186, 50)
Purchase_y      0.0
LogRevenue_y    0.0
dtype: float64
        LogRevenue_y    Purchase_y
0       1.574933e+09  1.574933e+09
1       1.574933e+09  1.574933e+09
2       1.574933e+09  1.574933e+09
3       1.574933e+09  1.574933e+09
4       1.574933e+09  1.574933e+09
...              ...           ...
377181  1.574933e+09  1.574933e+09
377182  1.574933e+09  1.574933e+09
377183  1.574933e+09  1.574933e+09
377184  1.574933e+09  1.574933e+09
377185  1.574933e+09  1.574933e+09

[377186 rows x 2 columns]
Iteration Done
Iteration: 1
Date Range 20160901 20170215
# of Visitors 371966
shape of training (459384, 10)
input shape: (459384, 10)
Splitting Json Done!! final shape: (459384, 55)
Final Shape (459384, 55)
# of Visitors 371966



data munge done!!
Purchase
0    370255
1      1711
dtype: int64


date range 20170401 20170531
input shape: (132497, 10)
Splitting Json Done!! final shape: (132497, 55)



final file characteristics
(371966, 50)
Purchase_y      0.0
LogRevenue_y    0.0
dtype: float64
        LogRevenue_y    Purchase_y
0       1.574933e+09  1.574933e+09
1       1.574933e+09  1.574933e+09
2       1.574933e+09  1.574933e+09
3       1.574933e+09  1.574933e+09
4       1.574933e+09  1.574933e+09
...              ...           ...
377181  1.574933e+09  1.574933e+09
377182  1.574933e+09  1.574933e+09
377183  1.574933e+09  1.574933e+09
377184  1.574933e+09  1.574933e+09
377185  1.574933e+09  1.574933e+09

[377186 rows x 2 columns]
Iteration Done
Iteration: 2
Date Range 20161001 20170315
# of Visitors 365362
shape of training (450592, 10)
input shape: (450592, 10)


KeyboardInterrupt: 

In [13]:
training = mungemunge(20180501, 20181016)
train1 = data_munge(training)
print(train1.groupby('Purchase').size())
train1.to_csv('final_data/train.csv', index = False)

Date Range 20180501 20181015
# of Visitors 296530
shape of training (401589, 10)
input shape: (401589, 10)
Splitting Json Done!! final shape: (401589, 56)
Final Shape (401589, 56)
# of Visitors 296530



data munge done!!
Purchase
0    292374
1      4156
dtype: int64


In [92]:
training = mungemunge(20170701, 20171215)
train1 = data_munge(training)
print(train1.groupby('Purchase').size())
train1.to_csv('final_data/train3.csv', index = False)

min date 20170701
max date 20171215
# of Visitors 384946
shape of training (496743, 10)
Load Dataframe to expand JSON
input shape: (496743, 10)
device
geoNetwork
totals
trafficSource
51.910603046417236
final shape: (496743, 56)
Final Shape (496743, 56)
# of Visitors 384946
done!!!



27.152862071990967
Purchase
0    394672
1      1138
dtype: int64


In [29]:
training = mungemunge(20180501, 20181015)
train1 = data_munge(training)
print(train1.groupby('Purchase').size())
train1.to_csv('final_data/test_new.csv', index = False)

Date Range 20180501 20181015
# of Visitors 296530
shape of training (401589, 10)
input shape: (401589, 10)
Splitting Json Done!! final shape: (401589, 56)
Final Shape (401589, 56)
# of Visitors 296530



data munge done!!
Purchase
0    292374
1      4156
dtype: int64


In [60]:
training = mungemunge(20180501, 20181015)
train1 = data_munge(training)
print(train1.groupby('Purchase').size())
train1.to_csv('final_data/test.csv', index = False)

min date 20180501
max date 20181015
# of Visitors 296530
shape of training (401589, 10)
Load Dataframe to expand JSON
input shape: (401589, 10)
device
geoNetwork
totals
trafficSource
47.28747224807739
final shape: (401589, 56)
Final Shape (401589, 56)
# of Visitors 296530
done!!!



(296530, 3)
(296530, 4)
(296530, 12)
(296530, 18)
(296530, 34)
(296530, 48)
Purchase
0    292374
1      4156
dtype: int64


In [23]:
def target_variable(filename, from_date, to_date):    
    df = test[(test['date']>=from_date) & (test['date']<=to_date)]
    print("min date",min(df['date']))
    print("max date",max(df['date']))
    
    df = load_df(df)
    df_target = df[['fullVisitorId', 'totals.transactionRevenue']]
    df_target['totals.transactionRevenue'] = df_target['totals.transactionRevenue'].astype(float)
    df_target = df_target.fillna(0)
    df_target = df_target.groupby('fullVisitorId').agg({'totals.transactionRevenue':'sum'})
    df_target['LogRevenue'] = np.log(df_target['totals.transactionRevenue'].astype(float) + 1)
    df_target['Purchase'] = df_target['totals.transactionRevenue'].swifter.apply(flag)
    df_target = df_target[['LogRevenue', 'Purchase']].reset_index()
    df_target.columns = ['fullVisitorId', 'LogRevenue_y', 'Purchase_y']
    
    
    current_df = pd.read_csv('final_data/'+filename, sep = ',')
    print()
    final = current_df.merge(df_target, on = 'fullVisitorId', how = 'left')
    final['LogRevenue_y'] = np.where(final['LogRevenue_y'].isnull(), 0, final['LogRevenue_y'])
    final['Purchase_y'] = np.where(final['Purchase_y'].isnull(), 0, 1)
    print('writing final file!')
    final.to_csv('final/'+filename, sep = ',', index = False)
    print('done')
    return final

In [133]:
final1 = target_variable('train1.csv', 20170301, 20170431)
final2 = target_variable('train2.csv', 20170816, 20171015)
final3 = target_variable('train3.csv', 20180201, 20180331)
final4 = target_variable('train4.csv', 20180616, 20180815)

min date 20170301
max date 20170430
input shape: (137057, 10)
device
geoNetwork
totals
trafficSource
12.51340913772583
final shape: (137057, 55)



writing final file!
done
min date 20170816
max date 20171015
input shape: (184346, 10)
device
geoNetwork
totals
trafficSource
19.11940288543701
final shape: (184346, 56)



writing final file!
done
min date 20180201
max date 20180331
input shape: (177093, 10)
device
geoNetwork
totals
trafficSource
18.600932359695435
final shape: (177093, 56)



writing final file!
done


ValueError: min() arg is an empty sequence

In [25]:
final4 = target_variable('train4.csv', 20180616, 20180815)

min date 20180616
max date 20180815
input shape: (137420, 10)
device
geoNetwork
totals
trafficSource
13.815101623535156
final shape: (137420, 56)




writing final file!
done


In [113]:
current_df = pd.read_csv('final/train1.csv', sep = ',')

In [115]:
print(current_df.shape)
current_df[['LogRevenue_y','Purchase_y']].sum(axis = 0)

(377186, 50)


LogRevenue_y    0.0
Purchase_y      0.0
dtype: float64

In [117]:
a = current_df[['LogRevenue_y','Purchase_y']]
a[a['Purchase_y'] == 1]

,LogRevenue_y,Purchase_y


In [61]:
t = pd.read_csv('final_data/test.csv')
t.shape

(296530, 48)

In [62]:
t[t['fullVisitorId'] == 450284137157873]

,fullVisitorId,LogRevenue,Purchase,visits,(Other),Affiliates,Direct,Display,Organic Search,Paid Search,...,trafSrc_isTrueDir_isNULL,trafSrc_isTrueDir_True,trafSrc_kw_others,trafSrc_kw_isNULL,trafSrc_medium_org,trafSrc_medium_ref,trafSrc_medium_isNULL,trafSrc_medium_cpc,trafSrc_medium_aff,trafSrc_medium_cpm
13,450284137157873,0.0,0,5,0,0,1,0,2,0,...,2,0,2,0,2,2,1,0,0,0


In [63]:
t.head()

,fullVisitorId,LogRevenue,Purchase,visits,(Other),Affiliates,Direct,Display,Organic Search,Paid Search,...,trafSrc_isTrueDir_isNULL,trafSrc_isTrueDir_True,trafSrc_kw_others,trafSrc_kw_isNULL,trafSrc_medium_org,trafSrc_medium_ref,trafSrc_medium_isNULL,trafSrc_medium_cpc,trafSrc_medium_aff,trafSrc_medium_cpm
0,18966949534117,0.0,0,1,0,0,0,0,1,0,...,1,0,1,0,1,0,0,0,0,0
1,39738481224681,0.0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,73585230191399,0.0,0,1,0,0,0,0,1,0,...,1,0,1,0,1,0,0,0,0,0
3,87588448856385,0.0,0,1,0,0,0,0,1,0,...,1,0,1,0,1,0,0,0,0,0
4,149787903119437,0.0,0,1,0,0,0,0,1,0,...,1,0,1,0,1,0,0,0,0,0


In [171]:
train0 = pd.read_csv('final/train0.csv')
train1 = pd.read_csv('final/train1.csv')
train2 = pd.read_csv('final/train2.csv')
train3 = pd.read_csv('final/train3.csv')
train4 = pd.read_csv('final/train4.csv')
train5 = pd.read_csv('final/train5.csv')
train6 = pd.read_csv('final/train6.csv')
train7 = pd.read_csv('final/train7.csv')
train8 = pd.read_csv('final/train8.csv')
train9 = pd.read_csv('final/train9.csv')
train10 = pd.read_csv('final/train10.csv')
train11 = pd.read_csv('final/train11.csv')
train12 = pd.read_csv('final/train12.csv')

In [207]:
training = pd.concat([train0, train1, train2, train3, train4, train5, train6, train7, train8, train9, train10, train11])

In [208]:
training.shape

(3987993, 50)

In [216]:
training[['Purchase_y', 'LogRevenue_y']].sum()

Purchase_y        6.000000
LogRevenue_y    104.876985
dtype: float64

In [217]:
training[training['Purchase_y'] == 1]

,fullVisitorId,LogRevenue,Purchase,visits,(Other),Affiliates,Direct,Display,Organic Search,Paid Search,...,trafSrc_kw_others,trafSrc_kw_isNULL,trafSrc_medium_org,trafSrc_medium_ref,trafSrc_medium_isNULL,trafSrc_medium_cpc,trafSrc_medium_aff,trafSrc_medium_cpm,LogRevenue_y,Purchase_y
127469,4300826865196193354,0.0,0,17,0,0,0,0,17,0,...,1,15,1,0,0,1,0,0,16.906098,1.0
150899,5097087346105463693,0.0,0,8,0,0,0,1,7,0,...,3,5,3,0,0,0,0,0,17.117992,1.0
162026,5305823939074339121,0.0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,18.119305,1.0
135190,4099631678878053818,0.0,0,19,0,0,2,0,0,0,...,2,17,1,1,1,1,0,0,16.587474,1.0
184933,5211936831694160345,0.0,0,1,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,17.686503,1.0
283606,7367993774068915456,0.0,0,3,0,0,3,0,0,0,...,1,2,1,0,1,0,1,0,18.459613,1.0
